In [5]:
PATH = "/Users/joregan/Playing/hu-tts/hungarian-single-speaker-tts"

In [ ]:
!pip install datasets librosa

In [ ]:
from pathlib import Path
import datasets

In [6]:
path = Path(PATH)
outdir = path / "data"
transcript = path / "transcript.txt"

In [40]:
def clean_text(text, norm = False):
    if text.endswith(" -"):
        text = text[:-2]
        text = text.replace(".S ", ". S ")
    if norm:
        text = text.replace(". ", " ")
        if len(text) > 1 and text[-1] in [".", "!", "?", ":"]:
            text = text[:-1]
        text = text.replace(" -, ", " ")
        text = text.replace(" - ", " ")
        text = text.replace(". ", " ")
        text = text.replace(", ", " ")
        text = text.replace("? ", " ")
        text = text.replace(": ", " ")
        text = text.replace("! ", " ")
        text = text.lower()
    return text

In [41]:
data = []
with open(str(transcript)) as ts:
    for line in ts.readlines():
        parts = line.strip().split("|")
        parts[1] = clean_text(parts[1])
        parts[2] = clean_text(parts[2], True)
        data.append(parts)

In [42]:
def data_gen():
    for i in data:
        filepath = i[0]
        fileid = filepath.split("/")[1].replace(".wav", "")
        text = i[1]
        lightly_cleaned = clean_text(text)
        fully_cleaned = clean_text(text, True)
        yield {
            "id": fileid,
            "audio": str(path / filepath),
            "original_text": lightly_cleaned,
            "text": fully_cleaned,
            "duration": float(i[3])
        }

In [43]:
from datasets import Dataset, Audio
dataset = Dataset.from_generator(data_gen).cast_column("audio", Audio())

Using custom data configuration default-1c789ef65a8a4cf0
/Users/joregan/opt/anaconda3/envs/hf/lib/python3.9/site-packages/datasets/builder.py:712: FutureWarning: 'use_auth_token' was deprecated in version 2.7.1 and will be removed in 3.0.0. Pass `use_auth_token` to the initializer/`load_dataset_builder` instead.
  warnings.warn(


Dataset generator downloaded and prepared to /Users/joregan/.cache/huggingface/datasets/generator/default-1c789ef65a8a4cf0/0.0.0. Subsequent calls will reuse this data.


In [44]:
dataset

Dataset({
    features: ['id', 'audio', 'original_text', 'text', 'duration'],
    num_rows: 4515
})

In [45]:
dataset[0]

{'id': 'egri_csillagok_0000',
 'audio': {'path': '/Users/joregan/Playing/hu-tts/hungarian-single-speaker-tts/egri_csillagok/egri_csillagok_0000.wav',
  'array': array([-0.0022583 , -0.00192261, -0.00180054, ..., -0.00079346,
         -0.00091553, -0.00082397], dtype=float32),
  'sampling_rate': 22050},
 'original_text': 'Az erdőben jártak, patakra találtak. A nap tüzesen sütött. A víz tetszett nekik.',
 'text': 'az erdőben jártak patakra találtak a nap tüzesen sütött a víz tetszett nekik',
 'duration': 6.75}